### 1. Use Splinter and/or Beautiful Soup to scrape mercadolibre.com.mx or soundcloud.com or tripadvisor.com. You must programmatically click a link from the front page to go to another page of interest to you and then select at least 5 items from that page. The 5 items must be stored in a data structure that makes sense (dictionary, list, etc).

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import csv
import re

url = 'https://www.mercadolibre.com.mx/'
page=requests.get(url)
soup = bs(page.text,'html.parser')

page

<Response [200]>

In [2]:
print(soup.title)

<title>Mercado Libre México - Envíos Gratis en el día</title>


In [3]:
for i in soup.find_all('a', href = True):
    print(i['href'])
    if "www.mercadolibre.com.mx/ofertas" in i['href']:
        nextpage = requests.get(i['href'])
        nextsoup = bs(nextpage.text, 'html.parser')
        
        print("next url title: ", nextsoup.find('title').string)

#root-app
//www.mercadolibre.com.mx
https://www.mercadolibre.com/jms/mlm/lgz/login?platform_id=ML&go=https%3A%2F%2Fwww.mercadolibre.com.mx%2F&loginType=explicit
https://www.mercadolibre.com.mx/registration?confirmation_url=https%3A%2F%2Fwww.mercadolibre.com.mx%2F
https://www.mercadolibre.com.mx#menu-user
https://www.mercadolibre.com.mx/ofertas#menu-user
next url title:  Ofertas | Mercadolibre México
https://www.mercadolibre.com.mx/gz/home/navigation#menu-user
https://www.mercadolibre.com.mx/ayuda#menu-user
https://www.mercadolibre.com.mx/supermercado#menu-user
https://www.mercadolibre.com.mx/c/ropa-bolsas-y-calzado#menu-user
https://www.mercadolibre.com.mx/mas-vendidos#menu-user
https://www.mercadolibre.com.mx/ofertas/compra-internacional?forceType=home&showDeal=true&splinterLab=admin#menu-user
next url title:  Compra internacional en Mercado Libre Mexico
https://www.mercadolibre.com.mx/tiendas-oficiales#menu-user
https://www.mercadolibre.com.mx/categorias#menu-user
https://www.mercado

Looks loke most of the next url on pages that contain the "www.mercadolibre.com.mx/ofertas" is directing back to the main page Ofertas. After examining the page closely, it turnes out the product items are on pages of the format: "www.mercadolibre.com.mx/ofertas?page=number". We can dynamically click through these a scrape information like description, price and other.

In [4]:
seed_url = 'https://www.mercadolibre.com.mx/ofertas?page={}'

def scrape(soup):
    # Find the elements of the article tag
    articles = soup.find_all('div', class_='promotion-item__container')

    # Iterate over each article tag
    for article in articles:
        price = article.find('span', class_='promotion-item__price').text
        title = article.find('p', class_='promotion-item__title')
        name = title.text
        price_dict = {'item':name,'Price':price}

        # Invoke the write_to_csv function
        write_to_csv([name, price])

def write_to_csv(list_input):
    # The scraped info will be written to a CSV
    try:
        with open("articles.csv", "a") as fopen:  # Open the csv file.
            csv_writer = csv.writer(fopen)
            csv_writer.writerow(list_input)
    except:
        return False

In [5]:
def browse_and_scrape(seed_url):

   # Page_number from the argument gets formatted in the URL & Fetched
    
    for i in range(10):
        formatted_url = seed_url.format(str(i))
        
        try:
            html_text = requests.get(formatted_url).text
            # Prepare the soup
            soup = bs(html_text, "html.parser")
            print(f"Now Scraping - {formatted_url}")
            scrape(soup)     # Invoke the scrape function
            time.sleep(3)

        except Exception as e:
            return e

In [6]:
if __name__ == "__main__":
    seed_url = 'https://www.mercadolibre.com.mx/ofertas?page={}'
    print("Web scraping has begun")
    result = browse_and_scrape(seed_url)
    if result == True:
        print("Web scraping is now complete!")
    else:
        print(f"Oops, That doesn't seem right!!! - {result}")

Web scraping has begun
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=0
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=1
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=2
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=3
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=4
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=5
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=6
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=7
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=8
Now Scraping - https://www.mercadolibre.com.mx/ofertas?page=9
Oops, That doesn't seem right!!! - None


In [7]:
import chardet
with open("articles.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [8]:
df = pd.read_csv("articles.csv", encoding = 'ISO-8859-1')
df

,Kit Ácido Hialurónico Serum+crema Día+crema Ojos+uv Defender,$ 54251
0,Taladro Atornillador Inalámbrico Sidsys 32pcs ...,$ 59728
1,Paquete De 2 Almohadas Spring Air Essence Peac...,$ 299
2,Colchón Emma Matrimonial | Memory Foam | Tecno...,"$ 7,359"
3,"300 Cubrebocas Tricapa, Mascarilla Termosellad...",$ 289
4,Nintendo Switch Oled 64gb Standard Color Blan...,"$ 9,349"
...,...,...
14266,Sierra Caladora Inalámbrica 20 V Klatter,"$ 1,017"
14267,Kit Garnier Fructis Hair Food Coco Rutina Comp...,$ 8894
14268,Tenis Para Mujer Nike Ad Comfort,"$ 1,084"
14269,Power Bank Pila Bateria Portatil 10000mah 1hora,$ 19799


### 2. Share with your group how web scraping worked on the website and link you chose.Compare it with others. List the site you chose and two observations about parsing the website. Indicate if this was the same or different for the sites chosen by others.

I chose mecadolibre after I failed with soundtrack. There I wanted to click through some songs and extract all comments, but the songs are not linked by hyperlinks and I am not exactly sure how they are linked there. In mercadolibre, once discovering how the ofertas page works and that they all have the same url, only a number differring, I could loop through the first few pages. I tried to do this increasing the page number by one after every scraping, but the problem is that if it reaches un unvalid number (210 or higher in this case), it redirects to the /ofertas homepage, so I never exited the loop. Maybe I should define that it shall exit if there are no items to scrape...